# Bio 산업 assignee와 compustat matching

### Compustat
- 2010년 상장사 재무정보
- nameprocessing 새로 필요

### matching 방법
- 이름 기준으로 matching (key가 딱히 없음)
- nameprocessing 결과로 1차 matching
- matching 실패한 assignee 중 num_patent 기준 상위 n%에 대해 manual matching
- match 후 key 부여 (index = key로 두면 될 듯)

### 마무리
- compustat_gvkey_numpat
- assignee_gvkey

In [1]:
#-*- coding: utf-8 -*-
import os
import pandas as pd

os.chdir('E:/apps')

### AG, BV, CORP, CO, kabushiki kaisha 처리 필요

def nameprocessing(s):
    s = s.lower()
    s = s.replace("-"," ")
    s = s.replace("&",'and')

    rm = ['.',',',';',"'",'kabushiki kaisha','kabushikikaisha','limited','company','corporation','incorporated']

    for i in rm:
        s = s.replace(i,"")

    sp=s.split()
    sp.reverse()

    abb=['inc','ltd','llc','pte','plc','kft','co','gmbh','corp','ag','bv','kg']

    for i in range(0,len(sp)):
        if sp[i] in abb:
            sp[i]=''
    
    sp.reverse()
    return ''.join(sp)

### 1. nameprocessing

In [2]:
com = pd.read_csv('compustat_10.csv')[['conm','gvkey']]
com['new_assignee'] = com['conm'].apply(nameprocessing)
com.iloc[0:5,:]

,conm,gvkey,new_assignee
0,ASM INTERNATIONAL NV,1166,asminternationalnv
1,HADERA PAPER LTD,1491,haderapaper
2,ATLAS CONS MINING & DEV CORP,1855,atlasconsmininganddev
3,BRITISH AMER TOBACCO PLC,1932,britishamertobacco
4,BANK LEUMI LE-ISRAEL BM,2018,bankleumileisraelbm


In [3]:
df = pd.read_csv('assignee_profile_BIO.csv')
df['new_assignee'] = df['assignee'].apply(nameprocessing)
df.to_csv('assignee_new_BIO.csv', index=False)
df.iloc[0:5,:]

,assignee,num_patent,new_assignee
0,BOSTON SCIENTIFIC SCIMED INC,1419,bostonscientificscimed
1,MONSANTO TECHNOLOGY LLC,1405,monsantotechnology
2,MEDTRONIC INC,1289,medtronic
3,PIONEER HI BRED INTERNATIONAL INC,1128,pioneerhibredinternational
4,CARDIAC PACEMAKERS INC,1071,cardiacpacemakers


### 2. matching

1. assignee를 new_assingee 기준으로 뭉치기
2. compustat과 new_assignee 기준으로 match

In [4]:
a=len(df)
df = df.groupby('new_assignee',as_index=False).sum()
df.sort_values('num_patent', ascending=False, inplace=True)

print a, len(df)

7413 7278


In [5]:
df = df.merge(com, on='new_assignee', how='left', sort=False)
df.drop_duplicates(inplace=True)   #I don't know why but there are some duplicates
df.to_csv('patent_compustat_key_BIO.csv',index=False)
print df['num_patent'].median(), df['num_patent'].mean()

2.0 10.8589074938


### 3. manual processing
- num_patent가 평균 보다 큰 기업의 경우(11개) matching 시도 (엑셀로) (전체의 80% 커버)
- median이 2.0이니 흠...

### 4. 마무리 = assignee join 후 쪼개기

__1. conm / gvkey 제작__
- gvkey 없는 애들 제거
- 있는 애들 뭉친 후 num_pat < 11인 애들 제거

In [16]:
#-*- coding: utf-8 -*-
import os
import pandas as pd

os.chdir('E:/apps')
df = pd.read_csv('compustat/patent_compustat_gvkey_BIO.csv')
df1 = df.loc[pd.isnull(df['gvkey'])==False]
print len(df), len(df1)

7283 1232


In [2]:
npat = df1[['gvkey','num_patent']]

npat = npat.groupby('gvkey',as_index=False).sum()
len(npat)

893

In [3]:
npat = npat.loc[npat['num_patent'] > 12]
len(npat)

325

In [4]:
npat = npat.merge(df1[['gvkey','conm']], on='gvkey', how='left', sort=False)
npat.drop_duplicates(inplace=True)

npat.sort_values(by='num_patent', ascending=False, axis=0, inplace=True)
npat.to_csv('compustat_gvkey_numpat_BIO.csv',index=False)

In [5]:
npat.iloc[0:20,]

,gvkey,num_patent,conm
255,25279.0,2868,BOSTON SCIENTIFIC CORP
119,7228.0,2644,MEDTRONIC PLC
81,6266.0,2409,JOHNSON & JOHNSON
51,4087.0,1709,DU PONT (E I) DE NEMOURS
0,1078.0,1697,ABBOTT LABORATORIES
474,140760.0,1521,MONSANTO CO
268,25648.0,1378,ROCHE HOLDING AG
524,177264.0,947,COVIDIEN PLC
397,101310.0,763,NOVARTIS AG
37,2403.0,691,BRISTOL-MYERS SQUIBB CO


__2. assignee / gvkey 제작__
- df1에서 npat에 쓴 325개 기업만 고르기
- new_assignee 기준으로 merge
- new_assingee 제거 후 assignee로 group by

In [17]:
df1 = df1[['gvkey','new_assignee']]

df1 = npat.merge(df1, on='gvkey', how='left', sort=False)
len(df1)

664

In [18]:
ass = pd.read_csv('assignee_new_BIO.csv')
df1 = df1.merge(ass, on='new_assignee', how='left', sort=False)
df1.drop_duplicates(inplace=True)
len(df1)

731

In [19]:
df1.iloc[0:20,:]

,gvkey,num_patent,conm,new_assignee,assignee
0,25279.0,2868,BOSTON SCIENTIFIC CORP,bostonscientificscimed,BOSTON SCIENTIFIC SCIMED INC
1,25279.0,2868,BOSTON SCIENTIFIC CORP,cardiacpacemakers,CARDIAC PACEMAKERS INC
2,25279.0,2868,BOSTON SCIENTIFIC CORP,bostonscientificneuromodulation,BOSTON SCIENTIFIC NEUROMODULATION CORPORATION
3,25279.0,2868,BOSTON SCIENTIFIC CORP,scimedlifesystems,SCIMED LIFE SYSTEMS INC
4,25279.0,2868,BOSTON SCIENTIFIC CORP,cameronhealth,CAMERON HEALTH INC
5,25279.0,2868,BOSTON SCIENTIFIC CORP,endovasculartechnologies,ENDOVASCULAR TECHNOLOGIES INC
6,25279.0,2868,BOSTON SCIENTIFIC CORP,bostonscientific,BOSTON SCIENTIFIC CORPORATION
7,25279.0,2868,BOSTON SCIENTIFIC CORP,bostonscientific,BOSTON SCIENTIFIC LIMITED
8,25279.0,2868,BOSTON SCIENTIFIC CORP,atritech,ATRITECH INC
9,25279.0,2868,BOSTON SCIENTIFIC CORP,remonmedicaltechnologies,REMON MEDICAL TECHNOLOGIES LTD


In [20]:
df1 = df1[['assignee','gvkey']]
df1.to_csv('assignee_gvkey_BIO.csv',index=False)